## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_selection import RFE, SelectFromModel, SelectKBest,f_regression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

## Loading Dataset

In [2]:
sales_df=pd.read_csv('/Users/Lovely_Ray/Desktop/data science/CTS/sales.csv')
sales_df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')
#Taking a look at the first 5 rows of the sales dataset
sales_df.head()

,transaction_id,timestamp,product_id,category,customer_type,unit_price,quantity,total,payment_type
0,a1c82654-c52c-45b3-8ce8-4c2a1efe63ed,2022-03-02 09:51:38,3bc6c1ea-0198-46de-9ffd-514ae3338713,fruit,gold,3.99,2,7.98,e-wallet
1,931ad550-09e8-4da6-beaa-8c9d17be9c60,2022-03-06 10:33:59,ad81b46c-bf38-41cf-9b54-5fe7f5eba93e,fruit,standard,3.99,1,3.99,e-wallet
2,ae133534-6f61-4cd6-b6b8-d1c1d8d90aea,2022-03-04 17:20:21,7c55cbd4-f306-4c04-a030-628cbe7867c1,fruit,premium,0.19,2,0.38,e-wallet
3,157cebd9-aaf0-475d-8a11-7c8e0f5b76e4,2022-03-02 17:23:58,80da8348-1707-403f-8be7-9e6deeccc883,fruit,gold,0.19,4,0.76,e-wallet
4,a81a6cd3-5e0c-44a2-826c-aea43e46c514,2022-03-05 14:32:43,7f5e86e6-f06f-45f6-bf44-27b095c9ad1d,fruit,basic,4.49,2,8.98,debit card


In [3]:
stock_df=pd.read_csv('/Users/Lovely_Ray/Desktop/data science/CTS/sensor_stock_levels.csv')
stock_df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')
#Taking a look at the first 5 rows of the stock dataset
stock_df.head()

,id,timestamp,product_id,estimated_stock_pct
0,4220e505-c247-478d-9831-6b9f87a4488a,2022-03-07 12:13:02,f658605e-75f3-4fed-a655-c0903f344427,0.75
1,f2612b26-fc82-49ea-8940-0751fdd4d9ef,2022-03-07 16:39:46,de06083a-f5c0-451d-b2f4-9ab88b52609d,0.48
2,989a287f-67e6-4478-aa49-c3a35dac0e2e,2022-03-01 18:17:43,ce8f3a04-d1a4-43b1-a7c2-fa1b8e7674c8,0.58
3,af8e5683-d247-46ac-9909-1a77bdebefb2,2022-03-02 14:29:09,c21e3ba9-92a3-4745-92c2-6faef73223f7,0.79
4,08a32247-3f44-4002-85fb-c198434dd4bb,2022-03-02 13:46:18,7f478817-aa5b-44e9-9059-8045228c9eb0,0.22


In [4]:
storage_df=pd.read_csv('/Users/Lovely_Ray/Desktop/data science/CTS/sensor_storage_temperature.csv')
storage_df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')
#Taking a look at the first 5 rows of the stock dataset
storage_df.head()

,id,timestamp,temperature
0,d1ca1ef8-0eac-42fc-af80-97106efc7b13,2022-03-07 15:55:20,2.96
1,4b8a66c4-0f3a-4f16-826f-8cf9397e9d18,2022-03-01 09:18:22,1.88
2,3d47a0c7-1e72-4512-812f-b6b5d8428cf3,2022-03-04 15:12:26,1.78
3,9500357b-ce15-424a-837a-7677b386f471,2022-03-02 12:30:42,2.18
4,c4b61fec-99c2-4c6d-8e5d-4edd8c9632fa,2022-03-05 09:09:33,1.38


## Data Pre-processing

In [5]:
#checking the no of columns and rows in the data
sales_df.shape

(7829, 9)

In [6]:
#checking the no of columns and rows in the data
stock_df.shape

(15000, 4)

In [7]:
#checking the no of columns and rows in the data
storage_df.shape

(23890, 3)

*** Now that we have our 3 datasets successfully loaded, we need to ensure that the data is clean. Data cleaning can be a very intense task, so for this exercise, we will focus just on ensuring that the correct datatypes are present for each column, and if not, correcting them.

*** We can use the .info() method to look at data types.

In [8]:
# Checking datatypes of the each features
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7829 entries, 0 to 7828
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  7829 non-null   object 
 1   timestamp       7829 non-null   object 
 2   product_id      7829 non-null   object 
 3   category        7829 non-null   object 
 4   customer_type   7829 non-null   object 
 5   unit_price      7829 non-null   float64
 6   quantity        7829 non-null   int64  
 7   total           7829 non-null   float64
 8   payment_type    7829 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 550.6+ KB


In [9]:
# Checking datatypes of the each features
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   15000 non-null  object 
 1   timestamp            15000 non-null  object 
 2   product_id           15000 non-null  object 
 3   estimated_stock_pct  15000 non-null  float64
dtypes: float64(1), object(3)
memory usage: 468.9+ KB


In [10]:
storage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23890 entries, 0 to 23889
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           23890 non-null  object 
 1   timestamp    23890 non-null  object 
 2   temperature  23890 non-null  float64
dtypes: float64(1), object(2)
memory usage: 560.0+ KB


*** Everything looks fine for the 3 datasets apart from the timestamp column in each dataset. Using the datetime function, let's convert this to the correct type for each dataset.

In [11]:
#changing 'time stamp'column type to datatime
sales_df['timestamp']=pd.to_datetime(sales_df['timestamp'], format='%Y-%m-%d %H:%M:%S')

In [12]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7829 entries, 0 to 7828
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   transaction_id  7829 non-null   object        
 1   timestamp       7829 non-null   datetime64[ns]
 2   product_id      7829 non-null   object        
 3   category        7829 non-null   object        
 4   customer_type   7829 non-null   object        
 5   unit_price      7829 non-null   float64       
 6   quantity        7829 non-null   int64         
 7   total           7829 non-null   float64       
 8   payment_type    7829 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 550.6+ KB


In [13]:
#changing 'time stamp'column type to datatime
stock_df['timestamp']=pd.to_datetime(stock_df['timestamp'], format='%Y-%m-%d %H:%M:%S')

In [14]:
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   15000 non-null  object        
 1   timestamp            15000 non-null  datetime64[ns]
 2   product_id           15000 non-null  object        
 3   estimated_stock_pct  15000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 468.9+ KB


In [15]:
#changing 'time stamp'column type to datatime
storage_df['timestamp']=pd.to_datetime(storage_df['timestamp'], format='%Y-%m-%d %H:%M:%S')
storage_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23890 entries, 0 to 23889
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           23890 non-null  object        
 1   timestamp    23890 non-null  datetime64[ns]
 2   temperature  23890 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 560.0+ KB


### Merging data

Currently we have 3 datasets. In order to include all of this data within a predictive model, we need to merge them together into 1 dataframe.

The problem statement is as follows:

“Can we accurately predict the stock levels of products, based on sales data and sensor data, 
on an hourly basis in order to more intelligently procure products from our suppliers.”

since we have to build a model that can predict on an hourly basis, looking at the data model, we can see that only column that we can use to merge the 3 datasets together is timestamp.

So, we must first transform the timestamp column in all 3 datasets to be based on the hour of the day, then we can merge the datasets together.

In [16]:
sales_df.head()

,transaction_id,timestamp,product_id,category,customer_type,unit_price,quantity,total,payment_type
0,a1c82654-c52c-45b3-8ce8-4c2a1efe63ed,2022-03-02 09:51:38,3bc6c1ea-0198-46de-9ffd-514ae3338713,fruit,gold,3.99,2,7.98,e-wallet
1,931ad550-09e8-4da6-beaa-8c9d17be9c60,2022-03-06 10:33:59,ad81b46c-bf38-41cf-9b54-5fe7f5eba93e,fruit,standard,3.99,1,3.99,e-wallet
2,ae133534-6f61-4cd6-b6b8-d1c1d8d90aea,2022-03-04 17:20:21,7c55cbd4-f306-4c04-a030-628cbe7867c1,fruit,premium,0.19,2,0.38,e-wallet
3,157cebd9-aaf0-475d-8a11-7c8e0f5b76e4,2022-03-02 17:23:58,80da8348-1707-403f-8be7-9e6deeccc883,fruit,gold,0.19,4,0.76,e-wallet
4,a81a6cd3-5e0c-44a2-826c-aea43e46c514,2022-03-05 14:32:43,7f5e86e6-f06f-45f6-bf44-27b095c9ad1d,fruit,basic,4.49,2,8.98,debit card


In [17]:
from datetime import datetime
def convert_timestamp_to_hourly(data: pd.DataFrame = None, column: str = None):
  dummy = data.copy()
  new_ts = dummy[column].tolist()
  new_ts = [i.strftime('%Y-%m-%d %H:00:00') for i in new_ts]
  new_ts = [datetime.strptime(i, '%Y-%m-%d %H:00:00') for i in new_ts]
  dummy[column] = new_ts
  return dummy

In [18]:
sales_df = convert_timestamp_to_hourly(sales_df, 'timestamp')
sales_df.head()

,transaction_id,timestamp,product_id,category,customer_type,unit_price,quantity,total,payment_type
0,a1c82654-c52c-45b3-8ce8-4c2a1efe63ed,2022-03-02 09:00:00,3bc6c1ea-0198-46de-9ffd-514ae3338713,fruit,gold,3.99,2,7.98,e-wallet
1,931ad550-09e8-4da6-beaa-8c9d17be9c60,2022-03-06 10:00:00,ad81b46c-bf38-41cf-9b54-5fe7f5eba93e,fruit,standard,3.99,1,3.99,e-wallet
2,ae133534-6f61-4cd6-b6b8-d1c1d8d90aea,2022-03-04 17:00:00,7c55cbd4-f306-4c04-a030-628cbe7867c1,fruit,premium,0.19,2,0.38,e-wallet
3,157cebd9-aaf0-475d-8a11-7c8e0f5b76e4,2022-03-02 17:00:00,80da8348-1707-403f-8be7-9e6deeccc883,fruit,gold,0.19,4,0.76,e-wallet
4,a81a6cd3-5e0c-44a2-826c-aea43e46c514,2022-03-05 14:00:00,7f5e86e6-f06f-45f6-bf44-27b095c9ad1d,fruit,basic,4.49,2,8.98,debit card


In [19]:
stock_df = convert_timestamp_to_hourly(stock_df, 'timestamp')
stock_df.head()

,id,timestamp,product_id,estimated_stock_pct
0,4220e505-c247-478d-9831-6b9f87a4488a,2022-03-07 12:00:00,f658605e-75f3-4fed-a655-c0903f344427,0.75
1,f2612b26-fc82-49ea-8940-0751fdd4d9ef,2022-03-07 16:00:00,de06083a-f5c0-451d-b2f4-9ab88b52609d,0.48
2,989a287f-67e6-4478-aa49-c3a35dac0e2e,2022-03-01 18:00:00,ce8f3a04-d1a4-43b1-a7c2-fa1b8e7674c8,0.58
3,af8e5683-d247-46ac-9909-1a77bdebefb2,2022-03-02 14:00:00,c21e3ba9-92a3-4745-92c2-6faef73223f7,0.79
4,08a32247-3f44-4002-85fb-c198434dd4bb,2022-03-02 13:00:00,7f478817-aa5b-44e9-9059-8045228c9eb0,0.22


In [20]:
storage_df = convert_timestamp_to_hourly(storage_df, 'timestamp')
storage_df.head()

,id,timestamp,temperature
0,d1ca1ef8-0eac-42fc-af80-97106efc7b13,2022-03-07 15:00:00,2.96
1,4b8a66c4-0f3a-4f16-826f-8cf9397e9d18,2022-03-01 09:00:00,1.88
2,3d47a0c7-1e72-4512-812f-b6b5d8428cf3,2022-03-04 15:00:00,1.78
3,9500357b-ce15-424a-837a-7677b386f471,2022-03-02 12:00:00,2.18
4,c4b61fec-99c2-4c6d-8e5d-4edd8c9632fa,2022-03-05 09:00:00,1.38


Now we have reduced the minutes and seconds to 00. next we will aggregate the datasets in order to combine rows which have the same value for timestamp.

For the sales data, we want to group the data by timestamp but also by product_id. we will also aggregate quantity.

In [21]:
#aggregating sales data with each row representing a unique combination of hour, sales of the unique product id.
sales_agg_data= sales_df.groupby(['timestamp', 'product_id']).agg({'quantity': 'sum'}).reset_index()
sales_agg_data.head()

,timestamp,product_id,quantity
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,3
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,3
2,2022-03-01 09:00:00,03a2557a-aa12-4add-a6d4-77dc36342067,3
3,2022-03-01 09:00:00,049b2171-0eeb-4a3e-bf98-0c290c7821da,7
4,2022-03-01 09:00:00,04da844d-8dba-4470-9119-e534d52a03a0,11


In [22]:
#grouping the data in the same way and aggregating the estimated_stock_pct.
#This shows us the average stock percentage of each product at unique hours.
stock_agg_data=stock_df.groupby(['timestamp', 'product_id']).agg({'estimated_stock_pct': 'mean'}).reset_index()
stock_agg_data.head()

,timestamp,product_id,estimated_stock_pct
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05


In [23]:
#for the storage data,we are grouping timestamp and aggregate the temperature.
storage_agg_data=storage_df.groupby(['timestamp']).agg({'temperature': 'mean'}).reset_index()
storage_agg_data.head()
#This gives us the average temperature of the storage facility where the product is stored by unique hours.

,timestamp,temperature
0,2022-03-01 09:00:00,-0.028850
1,2022-03-01 10:00:00,1.284314
2,2022-03-01 11:00:00,-0.560000
3,2022-03-01 12:00:00,-0.537721
4,2022-03-01 13:00:00,-0.188734


Now, we will merge our data. We will use the stock_agg_data as our base, and we will merge our other 2 tables onto this.

In [24]:
merged_df=stock_agg_data.merge(sales_agg_data, on=['timestamp', 'product_id'], how='left')
merged_df.head()

,timestamp,product_id,estimated_stock_pct,quantity
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,NaN
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,NaN
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,NaN


In [25]:
merged_df=merged_df.merge(storage_agg_data, on='timestamp', how='left')
merged_df.head()

,timestamp,product_id,estimated_stock_pct,quantity,temperature
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0,-0.02885
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0,-0.02885
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,NaN,-0.02885
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,NaN,-0.02885
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,NaN,-0.02885


In [26]:
merged_df.shape

(10845, 5)

In [27]:
#now we will chk the merged data
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10845 entries, 0 to 10844
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            10845 non-null  datetime64[ns]
 1   product_id           10845 non-null  object        
 2   estimated_stock_pct  10845 non-null  float64       
 3   quantity             3067 non-null   float64       
 4   temperature          10845 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 508.4+ KB


We can see some null values in the quantity column.so we have to treat this feature for model building purpose. We can assume that if there is a null value for this column, it represents that there were 0 sales of this product within this hour.hence,we will impute the null values with 0. however, we should verify this with the client to make sure we're not making any wroing assumptions by filling these null values with 0.

In [28]:
merged_df['quantity'] = merged_df['quantity'].fillna(0)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10845 entries, 0 to 10844
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            10845 non-null  datetime64[ns]
 1   product_id           10845 non-null  object        
 2   estimated_stock_pct  10845 non-null  float64       
 3   quantity             10845 non-null  float64       
 4   temperature          10845 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 508.4+ KB


In [29]:
#Checking duplicates
merged_df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
10840    False
10841    False
10842    False
10843    False
10844    False
Length: 10845, dtype: bool

now we have to add 2 more columns for model building purpose, viz. 'category' and 'unit price'.

In [30]:
product_cat=sales_df[['product_id', 'category']]
product_cat= product_cat.drop_duplicates()

product_price=sales_df[['product_id', 'unit_price']]
product_price=product_price.drop_duplicates()

In [31]:
product_cat.head()

,product_id,category
0,3bc6c1ea-0198-46de-9ffd-514ae3338713,fruit
1,ad81b46c-bf38-41cf-9b54-5fe7f5eba93e,fruit
2,7c55cbd4-f306-4c04-a030-628cbe7867c1,fruit
3,80da8348-1707-403f-8be7-9e6deeccc883,fruit
4,7f5e86e6-f06f-45f6-bf44-27b095c9ad1d,fruit


In [32]:
product_price.head()

,product_id,unit_price
0,3bc6c1ea-0198-46de-9ffd-514ae3338713,3.99
1,ad81b46c-bf38-41cf-9b54-5fe7f5eba93e,3.99
2,7c55cbd4-f306-4c04-a030-628cbe7867c1,0.19
3,80da8348-1707-403f-8be7-9e6deeccc883,0.19
4,7f5e86e6-f06f-45f6-bf44-27b095c9ad1d,4.49


In [33]:
merged_df=merged_df.merge(product_cat, on="product_id", how="left")
merged_df.head()

,timestamp,product_id,estimated_stock_pct,quantity,temperature,category
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0,-0.02885,kitchen
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0,-0.02885,vegetables
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,0.0,-0.02885,baby products
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,0.0,-0.02885,beverages
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,0.0,-0.02885,pets


In [34]:
merged_df=merged_df.merge(product_price, on="product_id", how="left")
merged_df.head()

,timestamp,product_id,estimated_stock_pct,quantity,temperature,category,unit_price
0,2022-03-01 09:00:00,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0,-0.02885,kitchen,11.19
1,2022-03-01 09:00:00,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0,-0.02885,vegetables,1.49
2,2022-03-01 09:00:00,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,0.0,-0.02885,baby products,14.19
3,2022-03-01 09:00:00,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,0.0,-0.02885,beverages,20.19
4,2022-03-01 09:00:00,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,0.0,-0.02885,pets,8.19


In [35]:
merged_df.shape

(10845, 7)

In [36]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10845 entries, 0 to 10844
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            10845 non-null  datetime64[ns]
 1   product_id           10845 non-null  object        
 2   estimated_stock_pct  10845 non-null  float64       
 3   quantity             10845 non-null  float64       
 4   temperature          10845 non-null  float64       
 5   category             10845 non-null  object        
 6   unit_price           10845 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 677.8+ KB


In [37]:
merged_df.duplicated().sum()

0

## Feature Engineering

since we have cleaned and merged our data, we have to transform this data so that the columns are in a suitable format for a machine learning model. In other terms, every column must be numeric. 

we will work with the timestamp column first. we can extract some useful features from this column for model building purpose.

In [38]:
merged_df['date_day'] = merged_df['timestamp'].dt.day
merged_df['date_dow'] = merged_df['timestamp'].dt.dayofweek
merged_df['hour'] = merged_df['timestamp'].dt.hour
merged_df.drop(columns=['timestamp'], inplace=True)
merged_df.head()

,product_id,estimated_stock_pct,quantity,temperature,category,unit_price,date_day,date_dow,hour
0,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0,-0.02885,kitchen,11.19,1,1,9
1,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0,-0.02885,vegetables,1.49,1,1,9
2,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,0.0,-0.02885,baby products,14.19,1,1,9
3,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,0.0,-0.02885,beverages,20.19,1,1,9
4,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,0.0,-0.02885,pets,8.19,1,1,9


now we will work on the category column and transform this column to numeric column.

In [39]:
merged_df=pd.get_dummies(merged_df, columns=['category'])
merged_df.head()

,product_id,estimated_stock_pct,quantity,temperature,unit_price,date_day,date_dow,hour,category_baby products,category_baked goods,...,category_meat,category_medicine,category_packaged foods,category_personal care,category_pets,category_refrigerated items,category_seafood,category_snacks,category_spices and herbs,category_vegetables
0,00e120bb-89d6-4df5-bc48-a051148e3d03,0.89,3.0,-0.02885,11.19,1,1,9,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01f3cdd9-8e9e-4dff-9b5c-69698a0388d0,0.14,3.0,-0.02885,1.49,1,1,9,0,0,...,0,0,0,0,0,0,0,0,0,1
2,01ff0803-ae73-4234-971d-5713c97b7f4b,0.67,0.0,-0.02885,14.19,1,1,9,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0363eb21-8c74-47e1-a216-c37e565e5ceb,0.82,0.0,-0.02885,20.19,1,1,9,0,0,...,0,0,0,0,0,0,0,0,0,0
4,03f0b20e-3b5b-444f-bc39-cdfa2523d4bc,0.05,0.0,-0.02885,8.19,1,1,9,0,0,...,0,0,0,0,1,0,0,0,0,0


In [40]:
merged_df.shape

(10845, 30)

In [41]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10845 entries, 0 to 10844
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   product_id                      10845 non-null  object 
 1   estimated_stock_pct             10845 non-null  float64
 2   quantity                        10845 non-null  float64
 3   temperature                     10845 non-null  float64
 4   unit_price                      10845 non-null  float64
 5   date_day                        10845 non-null  int64  
 6   date_dow                        10845 non-null  int64  
 7   hour                            10845 non-null  int64  
 8   category_baby products          10845 non-null  uint8  
 9   category_baked goods            10845 non-null  uint8  
 10  category_baking                 10845 non-null  uint8  
 11  category_beverages              10845 non-null  uint8  
 12  category_canned foods           

In [42]:
merged_df.drop(columns=['product_id'], inplace=True)
merged_df.head()

,estimated_stock_pct,quantity,temperature,unit_price,date_day,date_dow,hour,category_baby products,category_baked goods,category_baking,...,category_meat,category_medicine,category_packaged foods,category_personal care,category_pets,category_refrigerated items,category_seafood,category_snacks,category_spices and herbs,category_vegetables
0,0.89,3.0,-0.02885,11.19,1,1,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.14,3.0,-0.02885,1.49,1,1,9,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.67,0.0,-0.02885,14.19,1,1,9,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.82,0.0,-0.02885,20.19,1,1,9,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.05,0.0,-0.02885,8.19,1,1,9,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [43]:
merged_df.columns

Index(['estimated_stock_pct', 'quantity', 'temperature', 'unit_price',
       'date_day', 'date_dow', 'hour', 'category_baby products',
       'category_baked goods', 'category_baking', 'category_beverages',
       'category_canned foods', 'category_cheese',
       'category_cleaning products', 'category_condiments and sauces',
       'category_dairy', 'category_frozen', 'category_fruit',
       'category_kitchen', 'category_meat', 'category_medicine',
       'category_packaged foods', 'category_personal care', 'category_pets',
       'category_refrigerated items', 'category_seafood', 'category_snacks',
       'category_spices and herbs', 'category_vegetables'],
      dtype='object')

In [44]:
#Re-arranging the columns
final_df= merged_df[['quantity','unit_price', 'temperature', 
       'date_day', 'date_dow', 'hour', 'category_baby products',
       'category_baked goods', 'category_baking', 'category_beverages',
       'category_canned foods', 'category_cheese',
       'category_cleaning products', 'category_condiments and sauces',
       'category_dairy', 'category_frozen', 'category_fruit',
       'category_kitchen', 'category_meat', 'category_medicine',
       'category_packaged foods', 'category_personal care', 'category_pets',
       'category_refrigerated items', 'category_seafood', 'category_snacks',
       'category_spices and herbs', 'category_vegetables','estimated_stock_pct']]
final_df

,quantity,unit_price,temperature,date_day,date_dow,hour,category_baby products,category_baked goods,category_baking,category_beverages,...,category_medicine,category_packaged foods,category_personal care,category_pets,category_refrigerated items,category_seafood,category_snacks,category_spices and herbs,category_vegetables,estimated_stock_pct
0,3.0,11.19,-0.028850,1,1,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.89
1,3.0,1.49,-0.028850,1,1,9,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.14
2,0.0,14.19,-0.028850,1,1,9,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.67
3,0.0,20.19,-0.028850,1,1,9,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.82
4,0.0,8.19,-0.028850,1,1,9,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10840,4.0,4.99,-0.165077,7,0,19,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.50
10841,0.0,19.99,-0.165077,7,0,19,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.26
10842,3.0,6.99,-0.165077,7,0,19,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0.78
10843,3.0,14.99,-0.165077,7,0,19,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.92


now that feature engineering is complete, we will proceed towards model building.

## Model Building

In [45]:
#splitting the dataset
X=final_df.drop(columns='estimated_stock_pct',axis=1)
y=final_df['estimated_stock_pct']

In [46]:
print(X.shape)
print(y.shape)

(10845, 28)
(10845,)


In [47]:
#stardadizing the features using Standard scaler to bring features on a similar scale. 
sc = StandardScaler()
sc.fit_transform(X)

array([[ 1.16998954,  0.41770507,  0.28450808, ..., -0.22913726,
        -0.21799646, -0.212604  ],
       [ 1.16998954, -1.38192843,  0.28450808, ..., -0.22913726,
        -0.21799646,  4.70358027],
       [-0.50824032,  0.97429275,  0.28450808, ..., -0.22913726,
        -0.21799646, -0.212604  ],
       ...,
       [ 1.16998954, -0.36151768,  0.07481138, ..., -0.22913726,
        -0.21799646, -0.212604  ],
       [ 1.16998954,  1.12271613,  0.07481138, ..., -0.22913726,
        -0.21799646, -0.212604  ],
       [ 0.61057959, -0.69547029,  0.07481138, ..., -0.22913726,
        -0.21799646, -0.212604  ]])

### splitting the data into training and testing data

In [48]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2,random_state=2)

In [49]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(8676, 28) (2169, 28) (8676,) (2169,)


### Model training

In [50]:
svm=SVR(kernel='linear')
svm.fit(X_train,y_train)
rf=RandomForestRegressor(random_state=2)
rf.fit(X_train,y_train)
gr=GradientBoostingRegressor(random_state=2)
gr.fit(X_train,y_train)

GradientBoostingRegressor(random_state=2)

### Prediction on the test data

In [51]:
y_pred_svm=svm.predict(X_test)
y_pred_rf=rf.predict(X_test)
y_pred_gr=gr.predict(X_test)
df1=pd.DataFrame({'Actual':y_test,'svm':y_pred_svm,
                 'rf':y_pred_rf,'gr':y_pred_gr})

In [52]:
df1

,Actual,svm,rf,gr
5998,0.680,0.511870,0.319133,0.523184
2647,0.545,0.486588,0.451435,0.488978
8925,0.640,0.513512,0.393533,0.487570
5996,0.910,0.530890,0.347000,0.504726
2803,0.690,0.482487,0.579900,0.502994
...,...,...,...,...
9052,0.410,0.512691,0.757583,0.500750
2014,0.195,0.530499,0.504908,0.539733
4764,0.600,0.517133,0.365400,0.515343
3936,0.615,0.480296,0.354200,0.445606


### Model Evaluation

In [53]:
score_svm=metrics.r2_score(y_test,y_pred_svm)
score_rf=metrics.r2_score(y_test,y_pred_rf)
score_gr=metrics.r2_score(y_test,y_pred_gr)

In [54]:
print(score_svm,score_rf,score_gr)

-0.009305760775210059 -0.14045697971483495 -0.007949989033663396


In [55]:
#Adj R2 value
n = 10845
k = 28
Adj_score_svm=1 - (1 - score_svm)*(n-1) / (n-k-1)
Adj_score_rf=1 - (1 - score_rf)*(n-1) / (n-k-1)
Adj_score_gr=1 - (1 - score_gr)*(n-1) / (n-k-1)
print(Adj_score_svm,Adj_score_rf,Adj_score_gr)

-0.011918608528696195 -0.14340934615640433 -0.010559327023025533


In [56]:
#calculating MAE
s1=metrics.mean_absolute_error(y_test,y_pred_svm)
s2=metrics.mean_absolute_error(y_test,y_pred_rf)
s3=metrics.mean_absolute_error(y_test,y_pred_gr)
print(s1,s2,s3)

0.22661522642337678 0.23932909393363924 0.22651249717985303


### K Fold Validation

### Using Random forest model with all the features

In [57]:
X_ml=pd.concat([X_train, X_test])

In [58]:
y_ml = pd.concat([y_train, y_test])

In [59]:
all_ml = KFold(n_splits=20, shuffle=True, random_state=2)

In [60]:
score_all= cross_val_score(rf, X_ml, y_ml, cv=all_ml, scoring='r2')
score_all

array([-0.14971774, -0.21240047, -0.14420657, -0.19126309, -0.15205251,
       -0.09998629, -0.17703339, -0.20302345, -0.20057073, -0.12254167,
       -0.0554379 , -0.14608683, -0.18458408, -0.09494137, -0.16599567,
       -0.16721368, -0.15532619, -0.15889328, -0.15606346, -0.22598027])

In [61]:
np.round(score_all.mean()*100, 2)

-15.82

In [62]:
np.round(score_all.std()*100, 2)

4.08

In [65]:
score_all_mae= cross_val_score(rf, X_ml, y_ml, cv=all_ml, scoring='neg_mean_absolute_error')

In [66]:
print('Mean score over all folds: ', score_all_mae.mean())

Mean score over all folds:  -0.2364842489864897


### Using Support Vector model with all the features

In [67]:
score_all_svm= cross_val_score(svm, X_ml, y_ml, cv=all_ml, scoring='r2')
score_all_svm

array([-0.00335247, -0.00619728, -0.00023103, -0.00383945, -0.00611549,
       -0.00348478, -0.01534526, -0.02273866, -0.00439309, -0.00586216,
        0.00119747, -0.01457329, -0.01165632, -0.00903747,  0.00131332,
       -0.0159553 , -0.00470989,  0.00517672, -0.0075421 , -0.01434464])

In [68]:
score_all_mae_svm= cross_val_score(svm, X_ml, y_ml, cv=all_ml, scoring='neg_mean_absolute_error')

In [69]:
print('Mean score over all folds: ', score_all_mae_svm.mean())

Mean score over all folds:  -0.2230453733413201


### Using Gradient Boosting model with all the features

In [70]:
score_all_gr= cross_val_score(gr, X_ml, y_ml, cv=all_ml, scoring='r2')
score_all_gr

array([ 0.00376773, -0.01238414, -0.01463623, -0.00593478, -0.01255593,
        0.00286519, -0.00404278, -0.01878372, -0.01239111, -0.0039332 ,
        0.00059146, -0.01805035, -0.00748915, -0.00542335,  0.00557192,
       -0.01171574, -0.00050898, -0.00691079, -0.00149317, -0.01872173])

In [71]:
score_all_mae_gr= cross_val_score(gr, X_ml, y_ml, cv=all_ml, scoring='neg_mean_absolute_error')

In [72]:
print('Mean score over all folds: ', score_all_mae_gr.mean())

Mean score over all folds:  -0.22309224088002488


This is very interesting though. We can see that the mean absolute error (MAE) doesn't vary much for the above 3 models, Random Forest giving a mean MAE of 23.64. This is a good sign, it shows that the performance of the model is consistent across different random samples of the data, which is what we want. In other words, it shows a robust nature.

The MAE was chosen as a performance metric because it describes how closely the machine learning model was able to predict the exact value of estimated_stock_pct.

Even though the model is predicting robustly, this value for MAE is not so good, since the average value of the target variable is around 0.51, meaning that the accuracy as a percentage is around 50%. In an ideal world, we would want the MAE to be as low as possible. This is where the iterative process of machine learning comes in. At this stage, since we only have small samples of the data, we can report back to the client with these findings and recommend that the the dataset needs to be further engineered, or more datasets need to be added.